In [3]:
import pandas as pd
from util.util_data import *

## Metrics

- Date
- Outage_Color
- Service
- partial_outage_flag
- partial_outage_minutes
- major_outage_flag
- major_outage_minutes
- total_outage_minutes
- scaled_total_outage_minutes
- outage_impact_level 
      (int: The outage impact level, which can be one of the following:)
    - 0: Fully operational, no outages.
    - 1: Increasingly degraded performance (1 <= total_minutes <= 20).
    - 2: Increasingly partial outage (21 <= total_minutes <= 40).
    - 3: Increasingly major outage (41 <= total_minutes <= 60).
    - 4: Fully major outage (total_minutes > 60).
- Incident_Title
- incident_id
- incident_count

## clean data

In [91]:
def parse_outages(outages_list, outage_types=None):
    if outage_types is None:
        outage_types = ['Partial outage', 'Major outage']
    outages_dict ={}
    
    for outage_type in outage_types:
        outage_type = outage_type.lower().replace(' ', '_')
        outages_dict[outage_type + '_flag'] = int(0)
        outages_dict[outage_type + '_minutes'] = int(0)
        
    for outage in outages_list:
        outage_type = outage['Outage_Type'].lower().replace(' ', '_')
        outages_dict[outage_type + '_flag'] = int(1)
        outages_dict[outage_type + '_minutes'] = outage['Downtime (min)']
        
    return outages_dict

def get_outage_impact_level(total_minutes):
    """This function calculates the outage impact level based on the scaled total outage minutes."""
    if total_minutes == 0:
        return 0
    else:
        return min((total_minutes - 1) // 20 + 1, 4)

In [92]:
# load data

DATA_DIR = 'data/raw/uptime/2024-08-31'
DATA_LAYER = 'clean'
DATA_TYPE = 'outage'

df = read_data(DATA_DIR)

df = df.drop_duplicates()
df = df.reset_index(drop=True)
df['Date'] = pd.to_datetime(df['Date'])
df = load_json_column(df, 'Outages')
df = load_json_column(df, 'Incidents')
df['incident_count'] = df['Incidents'].apply(lambda x: len(x))

df


loading: data/raw/uptime/2024-08-31/console.anthropic.com/uptime_history.csv
loading: data/raw/uptime/2024-08-31/labs/uptime_history.csv
loading: data/raw/uptime/2024-08-31/character.ai/uptime_history.csv
loading: data/raw/uptime/2024-08-31/playground/uptime_history.csv
loading: data/raw/uptime/2024-08-31/api/uptime_history.csv
loading: data/raw/uptime/2024-08-31/chatgpt/uptime_history.csv
loading: data/raw/uptime/2024-08-31/api.anthropic.com/uptime_history.csv
loading: data/raw/uptime/2024-08-31/claude.ai/uptime_history.csv


,Date,Outages,Outage_Color,Incidents,Service,incident_count
0,2024-06-01,[],#2fcc66,[],console.anthropic.com,0
1,2024-06-02,[],#2fcc66,[],console.anthropic.com,0
2,2024-06-03,"[{'Outage_Type': 'Partial outage', 'Downtime (...",#e7c414,"[{'Incident_Title': 'Haiku Partial Outage', 'I...",console.anthropic.com,2
3,2024-06-04,"[{'Outage_Type': 'Partial outage', 'Downtime (...",#f0c40f,[{'Incident_Title': 'Partial Outage on claude....,console.anthropic.com,1
4,2024-06-05,[],#2fcc66,[],console.anthropic.com,0
...,...,...,...,...,...,...
5238,2023-08-27,[],#2fcc66,[],claude.ai,0
5239,2023-08-28,[],#2fcc66,[],claude.ai,0
5240,2023-08-29,[],#2fcc66,[],claude.ai,0
5241,2023-08-30,[],#2fcc66,[],claude.ai,0


In [93]:
# unnest outages
df['Outages'] = df['Outages'].apply(parse_outages)
df = unnest_dict(df, 'Outages')
df['outage_flag'] = df['major_outage_flag'] + df['partial_outage_flag']
df['total_outage_minutes'] = df['major_outage_minutes'] + df['partial_outage_minutes']
df['scaled_total_outage_minutes'] = df['major_outage_minutes'] + (df['partial_outage_minutes'] * 0.3)
df['outage_impact_level'] = df['scaled_total_outage_minutes'].apply(get_outage_impact_level).astype(int)

# add column provider
openai_service = ['api', 'chatgpt', 'labs', 'playground']
anthropic_service = ['claude.ai', 'api.anthropic.com', 'console.anthropic.com']
characterai_service = ['character.ai']
# Define a function to map services to providers
def get_provider(service):
    if service in openai_service:
        return 'openai'
    elif service in anthropic_service:
        return 'anthropic'
    elif service in characterai_service:
        return 'characterai'
    else:
        return 'unknown'
df['provider'] = df['Service'].apply(get_provider)

# save
write_partitioned_data(df, DATA_LAYER, DATA_TYPE, 'outage_unexploded.csv')

df
        

writing data to: data/clean/outage/2024-08-31/outage_unexploded.csv


,Date,Outage_Color,Incidents,Service,incident_count,partial_outage_flag,partial_outage_minutes,major_outage_flag,major_outage_minutes,outage_flag,total_outage_minutes,scaled_total_outage_minutes,outage_impact_level,provider
0,2024-06-01,#2fcc66,[],console.anthropic.com,0,0,0,0,0,0,0,0.0,0,anthropic
1,2024-06-02,#2fcc66,[],console.anthropic.com,0,0,0,0,0,0,0,0.0,0,anthropic
2,2024-06-03,#e7c414,"[{'Incident_Title': 'Haiku Partial Outage', 'I...",console.anthropic.com,2,1,59,0,0,1,59,17.7,1,anthropic
3,2024-06-04,#f0c40f,[{'Incident_Title': 'Partial Outage on claude....,console.anthropic.com,1,1,65,0,0,1,65,19.5,1,anthropic
4,2024-06-05,#2fcc66,[],console.anthropic.com,0,0,0,0,0,0,0,0.0,0,anthropic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5238,2023-08-27,#2fcc66,[],claude.ai,0,0,0,0,0,0,0,0.0,0,anthropic
5239,2023-08-28,#2fcc66,[],claude.ai,0,0,0,0,0,0,0,0.0,0,anthropic
5240,2023-08-29,#2fcc66,[],claude.ai,0,0,0,0,0,0,0,0.0,0,anthropic
5241,2023-08-30,#2fcc66,[],claude.ai,0,0,0,0,0,0,0,0.0,0,anthropic


In [94]:
services = ["api", "chatgpt", "labs", "playground", "claude.ai", "api.anthropic.com", "console.anthropic.com", "character.ai"]
for service in services:
    df_service = df[df['Service'] == service]
    print(df_service['major_outage_flag'].sum(), df_service['partial_outage_flag'].sum(), df_service['total_outage_minutes'].sum(), df_service['scaled_total_outage_minutes'].sum())


26 78 7891 3340.3
28 42 5185 2744.1
13 14 2821 1747.9
12 12 1636 1017.8999999999999
2 28 3017 982.8
0 25 1675 502.5
1 26 2032 662.1
17 15 3351 1877.5


## overview

In [6]:
df = pd.read_csv('data/clean/outage/2024-08-31/outage_unexploded.csv')
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,Outage_Color,Incidents,Service,incident_count,partial_outage_flag,partial_outage_minutes,major_outage_flag,major_outage_minutes,outage_flag,total_outage_minutes,scaled_total_outage_minutes,outage_impact_level,provider
0,2024-06-01,#2fcc66,[],console.anthropic.com,0,0,0,0,0,0,0,0.0,0,anthropic
1,2024-06-02,#2fcc66,[],console.anthropic.com,0,0,0,0,0,0,0,0.0,0,anthropic
2,2024-06-03,#e7c414,"[{'Incident_Title': 'Haiku Partial Outage', 'I...",console.anthropic.com,2,1,59,0,0,1,59,17.7,1,anthropic
3,2024-06-04,#f0c40f,[{'Incident_Title': 'Partial Outage on claude....,console.anthropic.com,1,1,65,0,0,1,65,19.5,1,anthropic
4,2024-06-05,#2fcc66,[],console.anthropic.com,0,0,0,0,0,0,0,0.0,0,anthropic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5238,2023-08-27,#2fcc66,[],claude.ai,0,0,0,0,0,0,0,0.0,0,anthropic
5239,2023-08-28,#2fcc66,[],claude.ai,0,0,0,0,0,0,0,0.0,0,anthropic
5240,2023-08-29,#2fcc66,[],claude.ai,0,0,0,0,0,0,0,0.0,0,anthropic
5241,2023-08-30,#2fcc66,[],claude.ai,0,0,0,0,0,0,0,0.0,0,anthropic


In [7]:
df_summary = df.groupby('Service').agg(
    start_date=('Date', 'min'),
    end_date=('Date', 'max'),
    # sum_days=('Date', 'count'),
    sum_months=('Date', lambda x: x.dt.to_period('M').nunique()),
    # sum_years=('Date', lambda x: x.dt.to_period('Y').nunique()),
    sum_total_outages_counts =('outage_flag', 'sum'),
    sum_major_outages_counts =('major_outage_flag', 'sum'),
    sum_partial_outages_counts =('partial_outage_flag', 'sum'),
    sum_total_outages_minutes =('total_outage_minutes', 'sum'),
    sum_scaled_total_outages_minutes =('scaled_total_outage_minutes', 'sum'),
    sum_incident_counts =('incident_count', 'sum')
).reset_index()

# df_summary.to_csv('plot/table/outage_summary.csv', index=False)

In [8]:
order = ['api', 'chatgpt', 'labs', 'playground', 'api.anthropic.com', 'claude.ai', 'console.anthropic.com', 'character.ai']
# order df by service
df_summary = df_summary.set_index('Service').loc[order].reset_index()
df_summary



,Service,start_date,end_date,sum_months,sum_total_outages_counts,sum_major_outages_counts,sum_partial_outages_counts,sum_total_outages_minutes,sum_scaled_total_outages_minutes,sum_incident_counts
0,api,2021-02-11,2024-08-31,43,104,26,78,7891,3340.3,242
1,chatgpt,2023-02-14,2024-08-31,19,70,28,42,5185,2744.1,157
2,labs,2023-02-21,2024-08-31,19,27,13,14,2821,1747.9,34
3,playground,2021-03-31,2024-08-31,42,24,12,12,1636,1017.9,36
4,api.anthropic.com,2023-07-11,2024-08-31,14,25,0,25,1675,502.5,80
5,claude.ai,2023-07-11,2024-08-31,14,30,2,28,3017,982.8,90
6,console.anthropic.com,2023-07-11,2024-08-31,14,27,1,26,2032,662.1,72
7,character.ai,2023-10-19,2024-08-31,11,32,17,15,3351,1877.5,41
